# DIY Covid-19 Dashboard

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the QMUL JupyterHub as well as in the Binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [9]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data and assign it as a dictionary to the ```jsondata``` variable (the code below will be hidden when the dashboard is rendered by Voila).

In [7]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("vaccinated.json", "rt") as INFILE:
    vacdata=json.load(INFILE)
with open("pcrtest.json", "rt") as INFILE:
    pcrdata=json.load(INFILE)
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and instead generate a dataframe with some random data

In [9]:
def vac_data(rawdata):
    datalist = rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    pcrdf=pd.DataFrame(index=index, columns=['death', 'first', 'second', 'third'])
    for entry in datalist: 
        date=parse_date(entry['date'])
        for column in ['death', 'first', 'second', 'third']:
            if pd.isna(pcrdf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                pcrdf.loc[date, column]=value
    pcrdf.fillna(0.0, inplace=True)
    return pcrdf


def pcr_data(rawdata):
    datalist = rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    pcrdf=pd.DataFrame(index=index, columns=['case', 'pcr', 'hospital'])
    for entry in datalist: 
        date=parse_date(entry['date'])
        for column in ['case', 'pcr', 'hospital']:
            if pd.isna(pcrdf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                pcrdf.loc[date, column]=value
    pcrdf.fillna(0.0, inplace=True)
    return pcrdf



# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in this cell as below:
pcrdf = pcr_data(pcrdata) # df is the dataframe for plotting
vacdf = vac_data(vacdata)

## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting (here, ```df```);
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

In this example, clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [13]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api(num):
    filter = ['areaType=nation', 'areaName=England']
    structure1 = {
    "date": "date",
    "death": "newDeaths28DaysByDeathDate",
    "first": "newPeopleVaccinatedFirstDoseByPublishDate",
    "second": "newPeopleVaccinatedSecondDoseByPublishDate",
    "third": "newPeopleVaccinatedThirdInjectionByPublishDate"
    }
    structure2 = {
        "date": "date",
        "case": "newCasesBySpecimenDate",
        "pcr": "newPCRTestsByPublishDate",
        "hospital": "newAdmissions",
    }
    if num == 1:
        api = Cov19API(filters=filters, structure=structure1)
    else:
        api = Cov19API(filters=filters, structure=structure2)
        
    return api.get_json()    

In [35]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api1_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api(1)
    global df
    df=vac_data(apidata)
    vac_graph()

def api2_button_callback(button):
    apidata=access_api(2)
    global df
    df=pcr_data(apidata)
    pcr_graph()

    
btn1_style = wdg.ButtonStyle()    
api1button=wdg.Button(
    description='Vaccinated Data',
    disabled=False,
)
btn1_style.button_color = 'lightblue'
btn1_style.font_weight = 'bold'       
btn1_style.border_color = 'gray' 
btn1_style.border_width = '2px'    
btn1_style.border_radius = '5px' 
api1button.style = btn1_style

btn2_style = wdg.ButtonStyle()
api2button=wdg.Button(
    description='PCR Data',
    disabled=False,
)
btn2_style.button_color = 'lightgreen'
btn2_style.font_weight = 'bold'       
btn2_style.border_color = 'white' 
btn2_style.border_width = '2px'    
btn2_style.border_radius = '5px' 
api2button.style = btn2_style

# remember to register your button callback function with the button
api1button.on_click(api1_button_callback)
api2button.on_click(api2_button_callback)

hbox = wdg.HBox([api1button, api2button])
display(hbox)

# run all cells before clicking on this button

## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [36]:
def plot_random_walk(walk):
    """ Our sample graph plotting function """
    df[walk].plot()
    plt.show() # important! update won't work properly without this

# a sample widget
whichwalk=wdg.Dropdown(
    options=['One', 'Two'],
    value='One',
    description='Walk no: ',
    disabled=False,
)

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=whichwalk.value
    if current==whichwalk.options[0]:
        other=whichwalk.options[1]
    else:
        other=whichwalk.options[0]
    whichwalk.value=other # forces the redraw
    whichwalk.value=current # now we can change it back
    
# connect the plotting function and the widget    
graph=wdg.interactive_output(plot_random_walk, {'walk': whichwalk})

# actually displays the graph
display(whichwalk, graph)

Dropdown(description='Walk no: ', options=('One', 'Two'), value='One')

Output()

In [38]:
pcrcols=wdg.SelectMultiple(
    options=['case', 'pcr', 'hospital'], # options available
    value=['case', 'pcr'], # initial value
    rows=3, # rows of the selection box
    description='pcr',
    disabled=False
)

def pcr_graph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        pcrdf.plot(y=list(graphcolumns))
        plt.show() # important - graphs won't update properly if this is missing
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
output=wdg.interactive_output(pcr_graph, {'graphcolumns': pcrcols})

display(pcrcols, output)

SelectMultiple(description='pcr', index=(0, 1), options=('case', 'pcr', 'hospital'), rows=3, value=('case', 'p…

Output()

In [39]:
vacdf = pd.read_pickle("vacdf.pkl")

vaccols=wdg.SelectMultiple(
    options=['death', 'first', 'second', 'third'], # options available
    value=['death', 'first', 'second', 'third'], # initial value
    rows=4, # rows of the selection box
    description='vaccination',
    disabled=False
)

def vac_graph(graphcolumns):
    # our callback function.
    ncols=len(graphcolumns)
    if ncols>0:
        vacdf.plot(y=list(graphcolumns))
        plt.show() # important - graphs won't update properly if this is missing
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in widget output    
output=wdg.interactive_output(vac_graph, {'graphcolumns': vaccols})

display(vaccols, output)

SelectMultiple(description='vaccination', index=(0, 1, 2, 3), options=('death', 'first', 'second', 'third'), r…

Output()

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 